In [2]:
# using PlotlyJS
using Plots
using PyCall
using Printf
using LsqFit
using LinearAlgebra
using Statistics
# using Flux
using Random
using DualNumbers
using ForwardDiff
# using Distributed
# using Base.Threads
# using Pkg
# using Optim
gr()  

Plots.GRBackend()

In [4]:
include("XY2.jl")

# Define Chebyshev polyonmials for construction of basis
chebyshevPolynomials::Vector{Function} = [
    x -> 1
    x -> x
    x -> 2*x^2 - 1
    x -> 4*x^3 - 3*x
    x -> 8*x^4 - 8*x^2 + 1
    x -> 16*x^5 - 20*x^3 + 5*x
    x -> 32*x^6 - 48*x^4 + 18*x^2 - 1
    x -> 64*x^7 - 112*x^5 + 56*x^3 - 7*x
    x -> 128*x^8 - 256*x^6 + 160*x^4 - 32*x^2 + 1
]
# Note Julia indexing starts from 1 so chebyshevPolynomials[n] is actually T_n-1 and so forth!

hartreeToWavenumberConversion::Float64 = 2.194746354e5
convertToRadians::Float64 = pi/180

# inputFileName::String = "CS2-AMES-fit-InitialFit"
# inputFileName::String = "CS2-AMES-fit-MorseFitted"
# inputFileName::String = "CS2-AMES-FitStructural"
# inputFileName::String = "CS2-AMES-rCSeFix"
# inputFileName::String = "CS2-AMES-Refit"
# inputFileName::String = "CS2-AMES-Refit-10thOrder"
# inputFileName::String = "CS2-AMES-fit-1D-Stretch"
inputFileName::String = "CS2-AMES-MorseUnfit"
# inputFileName::String = "CS2-AMES-fit-InitialFit-1D-StretchFittedMorse"
# inputFileName::String = "CS2-AMES-fit-InitialFit-1D-Alpha"
potentialInputFilePath::String = inputFileName*".inp"

keywords::Vector{String} = ["structural", "linear", "grid"]
inputBlocks::Vector{Vector{String}} = []
open(potentialInputFilePath, "r") do potentialInputFile::IOStream
    input::Vector{String} = readlines(potentialInputFile)
    addToBlock::Bool = false
    for keyword in keywords
        newBlock::Vector{String} = []
        for line in input
            if lowercase(line) == lowercase(keyword)
                addToBlock = true
            elseif lowercase(line) == "end"
                addToBlock = false
            end
            if addToBlock
                push!(newBlock, line)
            end
        end
        push!(inputBlocks, newBlock)
    end
end

structuralParameterBlock::Vector{String} = inputBlocks[1][2:end]
numberOfStructuralParameters::Int64 = size(structuralParameterBlock)[1] - 1
structuralParameterBlockSplit::Vector{Vector{SubString{String}}} = split.(structuralParameterBlock, r"\s+")
numberOfMorseParameters::Int64 = parse(Int64, structuralParameterBlockSplit[1][2])
numberOfEquilibriumParameters::Int64 = numberOfStructuralParameters - numberOfMorseParameters
numberOfModes::Int64 = size(structuralParameterBlockSplit[2])[1] - 3
numberOfAtoms::Int64 = (numberOfModes + 6)/3
numberOfStretches::Int64 = numberOfAtoms - 1
numberOfBondAngles::Int64 = numberOfAtoms - 2
numberOfDihedrals::Int64 = numberOfAtoms - 3

structuralParameters::Vector{Float64} = zeros(numberOfStructuralParameters)
structuralParameterLabels::Vector{String} = []
structuralPowers::Matrix{Int64} = zeros(numberOfStructuralParameters, numberOfModes)
structuralParametersOn::Vector{Int64} = zeros(numberOfStructuralParameters)
for i in 1:numberOfStructuralParameters
    structuralParameters[i] = parse(Float64, structuralParameterBlockSplit[i + 1][end])
    push!(structuralParameterLabels, String(structuralParameterBlockSplit[i + 1][1]))
    structuralPowers[i, :] = parse.(Int64, structuralParameterBlockSplit[i + 1][2:end-2])
    structuralParametersOn[i] = parse(Float64, structuralParameterBlockSplit[i + 1][end - 1])
end

structuralParameters::Vector{Float64} = convertAnglesToRadians(structuralParameters)

linearParameterBlock::Vector{String} = inputBlocks[2][2:end]
numberOfLinearParameters::Int64 = size(linearParameterBlock)[1]
linearParameterBlockSplit::Vector{Vector{SubString{String}}} = split.(linearParameterBlock, r"\s+")

linearParameters::Vector{Float64} = zeros(numberOfLinearParameters)
linearParameterLabels::Vector{String} = []
linearPowers::Matrix{Int64} = zeros(numberOfLinearParameters, numberOfModes)
linearPowersTotal::Vector{Int64} = zeros(numberOfLinearParameters)
linearParametersOn::Vector{Int64} = zeros(numberOfLinearParameters)

for i in 1:numberOfLinearParameters
    linearParameters[i] = parse(Float64, linearParameterBlockSplit[i][end])
    push!(linearParameterLabels, String(linearParameterBlockSplit[i][1]))
    linearPowers[i, :] = parse.(Int64, linearParameterBlockSplit[i][2:end-2])
    linearPowersTotal[i] = sum(linearPowers[i, :])
    linearParametersOn[i] = parse(Float64, linearParameterBlockSplit[i][end - 1])
end

# Reduce expansion order
maxExpansionOrder::Int64 = 6
indicesBelowExpansionOrderThreshold::Vector{Int64} = findall(<=(maxExpansionOrder), linearPowersTotal)
linearPowersTotal = linearPowersTotal[indicesBelowExpansionOrderThreshold]
linearParameters = linearParameters[indicesBelowExpansionOrderThreshold]
linearPowers = linearPowers[indicesBelowExpansionOrderThreshold, :] 
linearParametersOn = linearParametersOn[indicesBelowExpansionOrderThreshold]
numberOfLinearParameters = length(linearParameters)

allParameters::Vector{Float64} = vcat(structuralParameters, linearParameters)
allParameterLabels::Vector{String} = vcat(structuralParameterLabels, linearParameterLabels)
allPowers::Matrix{Int64} = vcat(structuralPowers, linearPowers)
allParametersOn::Vector{Int64} = vcat(structuralParametersOn, linearParametersOn)
numberOfParameters::Int64 = length(allParameters)
 
gridBlock::Vector{String} = inputBlocks[3][2:end]
numberOfGridPoints::Int64 = size(gridBlock)[1] 
gridBlockSplit::Vector{Vector{SubString{String}}} = split.(gridBlock, r"\s+")

grid::Matrix{Float64} = zeros(numberOfGridPoints, numberOfModes)
gridValenceCoordinates::Matrix{Float64} = zeros(numberOfGridPoints, numberOfModes)
energies::Vector{Float64} = zeros(numberOfGridPoints)

for i in 1:numberOfGridPoints
    grid[i, :]  = parse.(Float64, gridBlockSplit[i][1:numberOfModes])
    gridValenceCoordinates[i, :] = defineInternalCoordinates(grid[i, :])
    energies[i] = parse(Float64, gridBlockSplit[i][numberOfModes+1])
end

energyMaximum::Float64 = 3.0e4
indicesBelowEnergyThreshold::Vector{Int64} = findall(<(energyMaximum), energies)
energies = energies[indicesBelowEnergyThreshold]
gridValenceCoordinates = gridValenceCoordinates[indicesBelowEnergyThreshold, :]
grid = grid[indicesBelowEnergyThreshold, :]
numberOfGridPoints = length(energies)

referenceGeometry::Vector{Float64} = gridValenceCoordinates[1, :] 

# energies = energies.*hartreeToWavenumberConversion
# minimumEnergy::Float64 = minimum(energies)
println()
# @printf("%12.10f \n", minimumEnergy)
# energies = energies.-minimumEnergy

# Weight factor by Partridge and Schwenke
function computeWeightOfPoint(energy::Float64, energyThreshold=20000.0::Float64, energyMax=10000.0::Float64)::Float64
    weight::Float64 = (tanh(−0.0006*(energy - energyThreshold)) + 1.002002002)/2.002002002
    if energy > energyMax
        weight = weight/(0.0001*energy)
    else
        weight = weight/(0.0001*energyMax)
    end
    return weight
end

weights::Vector{Float64} = computeWeightOfPoint.(energies)
weightsSqrt::Vector{Float64} = sqrt.(weights)

function defineXiGrid(gridValenceCoordinates::Matrix{Float64}, structuralParameters)
    numberOfGridPoints::Int64 = size(gridValenceCoordinates)[1]
    T::DataType = eltype(structuralParameters)
    xiGrid::Matrix{T} = zeros(numberOfGridPoints, numberOfModes)
    for i in 1:numberOfGridPoints
        xiGrid[i, :] = defineXiCoordinates(gridValenceCoordinates[i, :], structuralParameters)
    end
    return xiGrid
end


# For range 1 to -1
# function rescaleXiGrid(xiGrid)
#     xiMaxima = maximum(xiGrid, dims=1)[1, :]
#     xiMinima = minimum(xiGrid, dims=1)[1, :]
#     xiGrid = (2*xiGrid .- (xiMaxima .+ xiMinima)')./(xiMaxima .- xiMinima)'
#     return xiGrid
# end

# For unit variance
function rescaleXiGrid(xiGrid)
    xistd = std(xiGrid, dims=1)[1, :]
    ximean = mean(xiGrid, dims=1)[1, :]
    xiGrid = (xiGrid .- (ximean)')./(xistd)'
    return xiGrid
end


function defineChebyshevBasis(xiGrid)
    numberOfGridPoints::Int64 = size(xiGrid)[1]
    T::DataType = eltype(xiGrid)
    xiMatrix::Matrix{T} = zeros(numberOfGridPoints, numberOfLinearParameters)
    for i in 1:numberOfGridPoints
        for j in 1:numberOfLinearParameters
            xiMatrix[i, j] = chebyshevPolynomials[linearPowers[j, 1]+1](xiGrid[i, 1])*chebyshevPolynomials[linearPowers[j, 2]+1](xiGrid[i, 2])*chebyshevPolynomials[linearPowers[j, 3]+1](xiGrid[i, 3])
            if linearPowers[j, 1] != linearPowers[j, 2]
                xiMatrix[i, j] += chebyshevPolynomials[linearPowers[j, 1]+1](xiGrid[i, 2])*chebyshevPolynomials[linearPowers[j, 2]+1](xiGrid[i, 1])*chebyshevPolynomials[linearPowers[j, 3]+1](xiGrid[i, 3])
            end
        end
    end
    return xiMatrix
end

function definePowersGrid(xiGrid)
    numberOfGridPoints::Int64 = size(xiGrid)[1]
    T::DataType = eltype(xiGrid)
    xiMatrix::Matrix{T} = zeros(numberOfGridPoints, numberOfLinearParameters)
    for i in 1:numberOfGridPoints
        for j in 1:numberOfLinearParameters
            xiMatrix[i, j] = xiGrid[i, 1]^linearPowers[j, 1]*xiGrid[i, 2]^linearPowers[j, 2]*xiGrid[i, 3]^linearPowers[j, 3]
            if linearPowers[j, 1] != linearPowers[j, 2]
                xiMatrix[i, j] += xiGrid[i, 2]^linearPowers[j, 1]*xiGrid[i, 1]^linearPowers[j, 2]*xiGrid[i, 3]^linearPowers[j, 3]
            end
        end
    end
    return xiMatrix
end

function solveLinearRegression(gridValenceCoordinates::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, structuralParameterIndex, structuralParameterForDerivative, λ::Float64 = 1e-7)
    T::DataType = typeof(structuralParameterForDerivative)
    numberOfGridPoints::Int64 = length(energies)
    structuralParametersInLossFunction::Vector{T} = copy(structuralParameters)
    structuralParametersInLossFunction[structuralParameterIndex] = structuralParameterForDerivative
    xiGrid = defineXiGrid(gridValenceCoordinates, structuralParametersInLossFunction)
    # xiGrid = rescaleXiGrid(xiGrid)
    # xiMatrix = defineChebyshevBasis(xiGrid)
    xiMatrix = definePowersGrid(xiGrid)
    weightsSqrt::Vector{Float64} = sqrt.(weights)
    weightedXiMatrix = xiMatrix .* weightsSqrt
    weightedEnergies = energies .* weightsSqrt
    tikhonovMatrix::Matrix{Float64} = λ.*Matrix(1I, numberOfLinearParameters, numberOfLinearParameters)
    linearParametersOfFit = (weightedXiMatrix'*weightedXiMatrix + tikhonovMatrix)\ (weightedXiMatrix'*weightedEnergies)
    computedEnergies::Vector{T} = xiMatrix*linearParametersOfFit
    residuals::Vector{T} = energies .- computedEnergies
    loss::T = (sum(weights.*residuals.^2) + sum(λ.*linearParametersOfFit.^2))/numberOfGridPoints
    return loss
end

function computeLossForwardDiff(gridValenceCoordinates::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, structuralParameters, λ::Float64 = 1e-7)
    T::DataType = eltype(structuralParameters)
    numberOfGridPoints::Int64 = length(energies)
    xiGrid = defineXiGrid(gridValenceCoordinates, structuralParameters)
    # xiGrid = rescaleXiGrid(xiGrid)
    # xiMatrix = defineChebyshevBasis(xiGrid)
    xiMatrix = definePowersGrid(xiGrid)
    computedEnergies::Vector{T} = xiMatrix*linearParameters
    residuals::Vector{T} = energies .- computedEnergies
    loss::T = (sum(weights.*residuals.^2) + sum(λ.*linearParameters.^2))/numberOfGridPoints
    return loss
end

function computeResdidualForwardDiff(gridValenceCoordinates::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, structuralParameters)
    T::DataType = eltype(structuralParameters)
    numberOfGridPoints::Int64 = length(energies)
    xiGrid = defineXiGrid(gridValenceCoordinates, structuralParameters)
    xiMatrix = definePowersGrid(xiGrid)
    weightedXiMatrix = xiMatrix .* weightsSqrt
    weightedEnergies = energies .* weightsSqrt
    # xiGrid = rescaleXiGrid(xiGrid)
    # xiMatrix = defineChebyshevBasis(xiGrid)
    computedEnergies::Vector{T} = weightedXiMatrix*linearParameters
    residuals::Vector{T} = weightedEnergies .- computedEnergies
    return residuals
end

function computeLoss(xiMatrix::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, λ::Float64 = 1e-7)
    numberOfGridPoints::Int64 = length(energies)
    computedEnergies::Vector{Float64} = xiMatrix*linearParameters
    residuals::Vector{Float64} = energies .- computedEnergies
    loss::Float64 = (sum(weights.*residuals.^2) + λ*sum(linearParameters.^2))/numberOfGridPoints
    return loss
end

function computeJacobianOfResiduals(xiMatrix::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64})
    numberOfGridPoints::Int64 = length(energies)
    weightedXiMatrix = xiMatrix .* weightsSqrt
    weightedEnergies = energies .* weightsSqrt
    # computedEnergies::Vector{Float64} = weightedXiMatrix*linearParameters
    jacobian::Matrix{Float64} = zeros(numberOfGridPoints, numberOfLinearParameters)
    for i in 1:numberOfGridPoints
        for j in 1:numberOfLinearParameters
            jacobian[i, j] = weightedEnergies[i] - weightedXiMatrix[i, j]
        end
    end
    return jacobian
end

function computeGradient(xiMatrix::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, λ::Float64 = 1e-7)
    numberOfGridPoints::Int64 = length(energies)
    computedEnergies::Vector{Float64} = xiMatrix*linearParameters
    residuals::Vector{Float64} = energies .- computedEnergies
    gradientVector::Vector{Float64} = 2*(-xiMatrix'*(weights.*residuals) .+ λ.*linearParameters)/numberOfGridPoints
    return gradientVector
end

function computeHessian(xiMatrix::Matrix{Float64}, energies::Vector{Float64}, weights::Vector{Float64}, λ::Float64 = 1e-7)
    numberOfGridPoints::Int64 = length(energies)
    weightsSqrt::Vector{Float64} = sqrt.(weights)
    weightedXiMatrix = xiMatrix .* weightsSqrt
    hessian::Matrix{Float64} = 2*(weightedXiMatrix'*weightedXiMatrix .+ λ*Matrix(1I, numberOfLinearParameters, numberOfLinearParameters))/numberOfGridPoints
    return hessian
end


structuralParameterBounds::Vector{Vector{Float64}} = [
    [1.55275000 - 0.2, 1.55275000 + 0.2],
    [1.0, 5.0],
]

xiGrid = defineXiGrid(gridValenceCoordinates, structuralParameters)
# xiGrid = rescaleXiGrid(xiGrid)
# xiMatrix::Matrix{Float64} = defineChebyshevBasis(xiGrid)
xiMatrix::Matrix{Float64} = definePowersGrid(xiGrid)

learningRateStructural::Float64 = 1.0
learningRateLinear::Float64 = 1.0
ϵ::Float64 = 1e-4 # Regulate weights for robust fitting
λ::Float64 = 1e-1 # Tikhonov Regularization parameter
epochs::Int64 = 200
for epoch in 1:epochs
    # structuralParameterGradient::Vector{Float64} = ForwardDiff.gradient(structuralParameters -> computeLossForwardDiff(gridValenceCoordinates, energies, weights, structuralParameters, λ), structuralParameters)
    # structuralParameterHessian::Matrix{Float64} = ForwardDiff.hessian(structuralParameters -> computeLossForwardDiff(gridValenceCoordinates, energies, weights, structuralParameters, λ), structuralParameters)
    # structuralParameters -= learningRateStructural*(structuralParameterHessian\structuralParameterGradient).*structuralParametersOn
    xiGrid = defineXiGrid(gridValenceCoordinates, structuralParameters)
    # xiGrid = rescaleXiGrid(xiGrid)
    # xiMatrix::Matrix{Float64} = defineChebyshevBasis(xiGrid)
    xiMatrix::Matrix{Float64} = definePowersGrid(xiGrid)
    # weightsSqrt::Vector{Float64} = sqrt.(weights)
    weightedXiMatrix = xiMatrix .* weightsSqrt
    weightedEnergies = energies .* weightsSqrt
    # linearParameters = (weightedXiMatrix'*weightedXiMatrix + λ.*Matrix(1I, numberOfLinearParameters, numberOfLinearParameters))\ (weightedXiMatrix'*weightedEnergies)
    # diag(computeHessian(xiMatrix, energies, weights, λ))
    # hessianLinear::Matrix{Float64} = computeHessian(xiMatrix, energies, weights, λ)
    # jacobianLinear::Vector{Float64} = computeGradient(xiMatrix, energies, weights, λ)
    jacobianLinear::Matrix{Float64} = computeJacobianOfResiduals(xiMatrix, energies, weights)
    linearParameters -= learningRateLinear*((weightedXiMatrix'*weightedXiMatrix .+ λ.*Matrix(1I, numberOfLinearParameters, numberOfLinearParameters)*linearPowersTotal.^2)\(-weightedXiMatrix'*(weightedEnergies .- weightedXiMatrix*linearParameters))).*linearParametersOn
    # linearParameters -= learningRateLinear.*(hessianLinear\jacobianLinear).*linearParametersOn
    # Q, R = qr(Diagonal(weightsSqrt)*xiMatrix)
    # linearParameters = R \ (Matrix(Q)' * (energies.*weightsSqrt))
    # weights = 1 ./ (residuals.^2 .+ ϵ)
    structuralParametersJacobian::Matrix{Float64} = ForwardDiff.jacobian(structuralParameters -> computeResdidualForwardDiff(gridValenceCoordinates, energies, weights, structuralParameters), structuralParameters)
    structuralParameters -= learningRateStructural*(structuralParametersJacobian'*structuralParametersJacobian)\(structuralParametersJacobian'*computeResdidualForwardDiff(gridValenceCoordinates, energies, weights, structuralParameters)).*structuralParametersOn
    structuralParameters = map((parameter, bound) -> clamp(parameter, bound[1], bound[2]), structuralParameters, structuralParameterBounds)
    residuals::Vector{Float64} = energies .- xiMatrix*linearParameters
    println("Epoch $(epoch), Loss = ", sum(weights.*residuals.^2)/numberOfGridPoints)
end


xiGrid = defineXiGrid(gridValenceCoordinates, structuralParameters)
# xiGrid = rescaleXiGrid(xiGrid)
xiMatrix::Matrix{Float64} = definePowersGrid(xiGrid)
# Q, R = qr(Diagonal(weightsSqrt)*testXiMatrix)
# linearParameters = R \ (Matrix(Q)' * (energies.*weightsSqrt))
# linearParameters
# println(solveLinearRegressionForForwardDiff(gridValenceCoordinates, energies, weights, 1, structuralParameters[1]))

sortedEnergyOrderingIndices::Vector{Int64} = sortperm(energies)
energies = energies[sortedEnergyOrderingIndices]
grid = grid[sortedEnergyOrderingIndices, :]
xiMatrix = xiMatrix[sortedEnergyOrderingIndices, :]

computedEnergies::Vector{Float64} = xiMatrix*linearParameters
residuals::Vector{Float64} = energies .- computedEnergies

structuralParameters = convertAnglesToDegrees(structuralParameters)

fittedParameters::Vector{Float64} = vcat(structuralParameters, linearParameters)
weights = computeWeightOfPoint.(energies)

open(inputFileName*".out", "w") do outputFile::IOStream
    println(outputFile, "New model:")
    for i in 1:numberOfParameters
        @printf(outputFile, "f   %4.0f %4.0f %4.0f  %12.12f\n", allPowers[i, 1], allPowers[i, 2], allPowers[i, 3], fittedParameters[i])
    end
    println(outputFile, )
    println(outputFile, "Grid of energies:")
    for i in 1:numberOfGridPoints
        @printf(outputFile, "%12.8f %12.8f %12.8f  %12.12f %12.12f %12.12f %12.12f\n", grid[i, 1], grid[i, 2], grid[i, 3],  energies[i], computedEnergies[i], residuals[i], weights[i])
    end
end
weightedRms::Float64 = sqrt(mean(weights.*residuals.^2))
println(weightedRms)


Epoch 1, Loss = 0.5031929528267824
Epoch 2, Loss = 0.5031929676461833
Epoch 3, Loss = 0.5031929692496738
Epoch 4, Loss = 0.50319297085307
Epoch 5, Loss = 0.5031929724567091
Epoch 6, Loss = 0.503192974060143
Epoch 7, Loss = 0.5031929756635716
Epoch 8, Loss = 0.5031929772669722
Epoch 9, Loss = 0.5031929788704352
Epoch 10, Loss = 0.5031929804739746
Epoch 11, Loss = 0.5031929820772897
Epoch 12, Loss = 0.5031929836807005
Epoch 13, Loss = 0.5031929852841825
Epoch 14, Loss = 0.5031929868877986
Epoch 15, Loss = 0.5031929884912771
Epoch 16, Loss = 0.5031929900946557
Epoch 17, Loss = 0.503192991698204
Epoch 18, Loss = 0.5031929933015475
Epoch 19, Loss = 0.5031929949049722
Epoch 20, Loss = 0.5031929965084071
Epoch 21, Loss = 0.5031929981118678
Epoch 22, Loss = 0.5031929997153118
Epoch 23, Loss = 0.5031930013189568
Epoch 24, Loss = 0.5031930029223826
Epoch 25, Loss = 0.5031930045258457
Epoch 26, Loss = 0.5031930061292076
Epoch 27, Loss = 0.5031930077327161
Epoch 28, Loss = 0.5031930093361198
Epoc

In [ ]:
Plots.scatter(energies, residuals,
    linewidth=2,
    legendfontsize=10,
    guidefontsize=14,
    tickfontsize=12,
    titlefontsize=16,
    labelfontsize=14,
    xlimits=(0,20000),
    ylimits=(-5, 5)
)

In [ ]:
Plots.scatter(energies, residuals,
    linewidth=2,
    legendfontsize=10,
    guidefontsize=14,
    tickfontsize=12,
    titlefontsize=16,
    labelfontsize=14,
)